# Importing

In [ ]:
# Cleanning
import pandas as pd
import numpy as np
from scipy.stats import mode

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Modelling
from haversine import haversine
from scipy.stats import boxcox
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import (StandardScaler, MinMaxScaler)
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import iqr
from scipy.stats import scoreatpercentile as pct
from sklearn.cluster import KMeans
from scipy.stats import norm
from scipy.stats import probplot
from math import radians, cos, sin, asin, sqrt
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
import xgboost
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score

# Define functions

## Importing Data

In [ ]:
def load_data(file, source_type="csv"):
    """
    

    Parameters
    ----------
    file : str
        Name of the file containing the data.
    source_type : str, optional
        The file type."csv" or "excel" accepted. The default is "csv".

    Returns
    -------
    file1 : Dataframe
        A DataFrame containing the data.

    """

    path = "Data/" + file

    try:

        read_f = eval(f'pd.read_{source_type}')
        file1 = read_f(path)
        return file1

    except AttributeError:

        print("Sorry, your input source_type is not supported. Try 'csv' or 'excel'.")

## Cleaning and preprocessing data

### Standardizing Headings

In [ ]:

def standard_headings(df):
    """
    

    Parameters
    ----------
    df : DataFrame
        The DataFrame which headings will be standardized.

    Returns
    -------
    df : DataFrame
        A DataFrame with standardized headings, i.e. lower case and " " replaced by "_".

    """

    heading = df.columns
    df.columns = [clabel.lower().replace(" ", "_") for clabel in heading]
    return df

### Transform yr_renovated and yr_built into one single column


In [ ]:
# Wrapper function (to use the block of code in a pipeline), only usable for this dataset
# Combine the renovation year (with many zeros, possibly missing values) and the construction year into one single column

def transform_renovated_built(df):

    yr_renovated = df["yr_renovated"]
    yr_built = df["yr_built"]
    yr_ren_built = yr_renovated.where(yr_renovated != 0, other = yr_built)
    age=  max(yr_ren_built) - yr_ren_built
    df["yr_built"] = age
    df = df.rename(columns={"yr_built":"age"})
    df = df.drop(columns=["yr_renovated"])   

    return df

### Removing specific rows


In [ ]:
# Wrapper function (to use the block of code in a pipeline), only usable for this dataset
# There is one record with 33 rooms that is obviously a typo

def remove_rows(df):

    df = df[df["bedrooms"] != 33]

    return df

## Data Transformation

### X/y split | Train/test split | Numerical Variables Scaling


In [ ]:

def my_transformations(x, y, test_size=0.2, numerical=True, scaler="standard", 
                       categorical=True, ordinal_dict=None, nominal_list=[]):
    """

    Parameters
    ----------
    x : DataFrame or Series
        Estimators
    y : DataFrame or Series
        Target Variable
    test_size : float, optional
        Size in fraction (0-1) of the test set. The default is 0.2.
    numerical : Boolean, optional
        Numerical Flag, True if the output DataFrame should contain numerical variables. 
        The default is True.
    scaler : String, optional
        The type of numerical scaling method to be used. Values accepted are 
        "standard" and "minmax". The default is "standard".
    categorical : Boolean, optional
        Categorical Flag, True if the output DataFrame should contain categorical variables.
        The default is True.
    ordinal_dict : Dictionary, optional
        Dictionary containing the rules for the ordinal encoding. The default is None.
    nominal_list : List, optional
        List containing the columns that require one-hot encoding . The default is [].

    Returns
    -------
    dict
        A dictionary containing two records, "train" and "test".
        These two are lists containing the transformed train and test set:
            - Scaled Numerical columns
            - Ordinal Encoded columns
            - One-Hot Encoded columns
            - Target Variable

    """

    # Train/Test Split

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=0)

    x_train = x_train.reset_index(drop=True)
    x_test = x_test.reset_index(drop=True)

    y_train = y_train.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)

    # Numerical variables - assign and scale them

    if numerical:

        x_train_num = x_train._get_numeric_data()
        x_test_num = x_test._get_numeric_data()

        if scaler is not None:

            if scaler == "minmax":

                scaler = MinMaxScaler()

            elif scaler == "standard":

                scaler = StandardScaler()

            else:

                print("The input scaling method is not supported. Please use 'standard' or 'minmax'.")
                return

            scaler.fit(x_train_num)
            output_train_num = pd.DataFrame(scaler.transform(x_train_num), columns=x_train_num.columns)
            output_test_num = pd.DataFrame(scaler.transform(x_test_num), columns=x_test_num.columns)

        else:

            output_train_num = x_train_num
            output_test_num = x_test_num
    else:

        output_train_num = "None"
        output_test_num = "None"

    # Categorical variables - assign and encode them

    if categorical:

        x_train_cat = x_train.select_dtypes(["category", "object"])
        x_test_cat = x_test.select_dtypes(["category", "object"])

        # Encoding - Ordinals

        if ordinal_dict is not None:

            x_train_cat_ord = x_train_cat[list(ordinal_dict.keys())]
            x_test_cat_ord = x_test_cat[list(ordinal_dict.keys())]

            categories = [t[1] for t in list(ordinal_dict.items())]

            ordinal_encoder = OrdinalEncoder(categories=categories)
            x_train_cat[x_train_cat_ord.columns] = pd.DataFrame(ordinal_encoder.fit_transform(x_train_cat_ord)
                                                                , columns=x_train_cat_ord.columns)
            output_train_cat_ord = x_train_cat.drop(nominal_list, axis=1)
            x_test_cat[x_test_cat_ord.columns] = pd.DataFrame(ordinal_encoder.fit_transform(x_test_cat_ord)
                                                              , columns=x_test_cat_ord.columns)
            output_test_cat_ord = x_test_cat.drop(nominal_list, axis=1)

        else:

            output_train_cat_ord = x_train_cat
            output_test_cat_ord = x_test_cat

        # Encoding - Nominals

        if len(nominal_list) != 0:

            output_train_cat_nom = pd.get_dummies(x_train_cat.loc[:, nominal_list], drop_first=True)
            output_test_cat_nom = pd.get_dummies(x_test_cat.loc[:, nominal_list], drop_first=True)

        else:

            output_train_cat_nom = "None"
            output_test_cat_nom = "None"

    else:

        output_train_cat_ord = "None"
        output_train_cat_nom = "None"
        output_test_cat_ord = "None"
        output_test_cat_nom = "None"

    return {"train": [output_train_num, output_train_cat_ord, output_train_cat_nom, y_train],
            "test": [output_test_num, output_test_cat_ord, output_test_cat_nom, y_test]}

### Normalization

In [ ]:
def var_normalization(df):
    """


    Parameters
    ----------
    df : DataFrame or Series
        Target Dataframe to be nomalized.

    Returns
    -------
    transformer : Sklearn Transformet Object
        Box-Cox Transformer Object.
    power_norm : DataFrame or Series
        Target DataFrame or Series Normalized using a box-cox transformation.

    """

    transformer = PowerTransformer(method="box-cox").fit(df.to_numpy().reshape(-1, 1))
    power_norm = transformer.transform(df.to_numpy().reshape(-1, 1))
    power_norm = pd.DataFrame(power_norm)[0]

    return transformer, power_norm

In [ ]:
# Wrapper function (to use the block of code in a pipeline), only usable for this dataset
# Normalization Tranform of price and sqft_living

def normalizations(df):

    df["price"] = np.log(df["price"])
    df["sqft_living"] = var_normalization(df["sqft_living"])[1]

    return df

### Location Engineered Feature


In [ ]:
# Wrapper function (to use the block of code in a pipeline), only usable for this dataset
# Engineered Feature to treat the location data as the distance of each house to two hot spots, bellevue and Seattle downtown

def dist_to(df):

    latlong = df.loc[:, ['lat', 'long']]
    latlong_tuple = list(zip(latlong.loc[:, 'lat'], latlong.loc[:, 'long']))
    latlong["lat_long"] = latlong_tuple

    dist_to_seattle = latlong.loc[:, 'lat_long'].apply(haversine, point2=(47.609395, -122.336283))
    dist_to_bellevue = latlong.loc[:, 'lat_long'].apply(haversine, point2=(47.616492, -122.188985))
    
    df.loc[:, 'dist_to_seattle'] = dist_to_seattle
    df.loc[:, 'dist_to_bellevue'] = dist_to_bellevue

    return df

## Regression Functions


### Performance Metrics


In [ ]:
def r2_adjusted(x, y, y_pred, r2=None):
    """
    

    Parameters
    ----------
    x : DataFrame or Serie
        Estimators
    y : DataFrame or Serie
        Observed Target Variable.
    y_pred : DataFrame or Serie
        Predicted Target Variable.
    r2 : Float, optional
        R2 value previously calculated. The default is None.

    Returns
    -------
    r2_adj : Float
        Adjusted R2 value.

    """

    if r2 is None:

        r2 = r2_score(y, y_pred)


    r2_adj = 1 - (1 - r2) * (len(y) - 1) / (len(y) - x.shape[1] - 1)

    return r2_adj

In [ ]:
def perf_regression(y_train, y_test, y_pred_train, y_pred_test, x_train, x_test):
    """
    

    Parameters
    ----------
    y_train : DataFrame or Series
        Observed Target Variable (train set).
    y_test : DataFrame or Series
        Observed Target Variable (test set).
    y_pred_train : DataFrame or Series
        Predicted Target Variable (train set).
    y_pred_test : DataFrame or Series
        Predicted Target Variable (test set).
    x_train : DataFrame or Series
        Estimators (train set).
    x_test : DataFrame or Series
        Estimators (test set).

    Returns
    -------
    performance : DataFrame
        Dataframe containing the performance metrics (MAE, MSE, RMSE, R2 and R2 adj).

    """

    performance = pd.DataFrame({'error_metric': ['mae', 'mse', 'rmse', 'r2', 'r2_adj'],
                                'train': [mae(y_train, y_pred_train),
                                          mse(y_train, y_pred_train),
                                          mse(y_train, y_pred_train, squared=False),
                                          r2_score(y_train, y_pred_train),
                                          r2_adjusted(x_train, y_train, y_pred_train)],
                                'test':  [mae(y_test, y_pred_test),
                                          mse(y_test, y_pred_test),
                                          mse(y_test, y_pred_test, squared=False),
                                          r2_score(y_test, y_pred_test),
                                          r2_adjusted(x_test, y_test, y_pred_test)]})

    performance = performance.set_index("error_metric")

    return performance

In [ ]:
def regression_plots(pred_data, set_id, fig_size=(15, 5)):
    """


    Parameters
    ----------
    pred_data : Dictionary
        Contains the observed and the predicted target variable.
    set_id : String
        "train" or "test".
    fig_size : Tuple, optional
        Figure size. The default is (15, 5).

    Returns
    -------
    None.

    """

    y = pred_data["y"]
    y_pred = pred_data["y_pred"]

    fig, axs = plt.subplots(1, 3, figsize=fig_size)

    sns.regplot(x="y", y="y_pred", data=pred_data, scatter_kws={"color": "red"}, line_kws={"color": "black"}, ax=axs[0])
    sns.histplot(y - y_pred, kde=True, ax=axs[1])
    axs[2].plot(y_pred, y - y_pred, "o")
    axs[2].plot(y_pred, np.zeros(len(y_pred)), linestyle='dashed')

    axs[0].set_title(f"{set_id}".capitalize() + " Set - Observed VS Predicted")
    axs[1].set_title(f"{set_id}".capitalize() + " Set - Histogram of the Residuals")
    axs[2].set_title("Residuals by Predicted")

    axs[1].set_xlabel(f"y_{set_id}" + " - y_pred")
    axs[2].set_xlabel("Predicted")
    axs[2].set_ylabel("Residuals")

### Flexible Method Regression


In [ ]:
def regression(x_train, y_train, x_test, y_test, model, cv=10, verbose=True, plot=True, y_transformer=None):
    """
    

    Parameters
    ----------
    x_train : DataFrame or Series
        Estimators (train set).
    y_train : DataFrame or Series
        Observed Target Variable (train set).
    x_test : DataFrame or Series
        Estimators (test set).
    y_test : DataFrame or Series
        Observed Target Variable (test set).
    model : Sklearn model object
        Model to be used for the prediction
    cv : Integer, optional
        Number of cross validation folds. The default is 10.
    verbose : Boolean, optional
        The default is True.
    plot : Boolean, optional
        The default is True.
    y_transformer : String or sklearn transformer object, optional
        The normalization transformer used on the target variable, if any. The default is None.

    Returns
    -------
    dict
        Results of the regression.
        - Fitted Model
        - Cross Validation Scores
        - Prediction results
        - Performance Metrics

    """
    
    # Model cross validation
    val_scores = cross_val_score(model, x_train, y_train, cv=cv)

    # Fit the model
    model.fit(x_train, y_train)

    # Predictions
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    if y_transformer is not None:

        if y_transformer != "log":

            y_train = pd.DataFrame(y_transformer.inverse_transform(np.array(y_train).reshape(-1, 1)))[0]
            y_pred_train = pd.DataFrame(y_transformer.inverse_transform(y_pred_train.reshape(-1, 1)))[0]

            y_test = pd.DataFrame(y_transformer.inverse_transform(np.array(y_test).reshape(-1, 1)))[0]
            y_pred_test = pd.DataFrame(y_transformer.inverse_transform(y_pred_test.reshape(-1, 1)))[0]

        elif y_transformer == "log":

            y_train = np.exp(y_train)
            y_pred_train = np.exp(y_pred_train)

            y_test = np.exp(y_test)
            y_pred_test = np.exp(y_pred_test)

    prediction_results = pd.DataFrame({"train": {"y": y_train, "y_pred": y_pred_train},
                                       "test": {"y": y_test, "y_pred": y_pred_test}})

    # Build the performance df
    performance_metrics = pd.DataFrame({"error_metric": [f'val_mean_score (k={cv})', f'val_std (k={cv})'],
                                        "train": [round(np.mean(val_scores), 3), round(np.std(val_scores), 3)],
                                        "test": ["-", "-"]})
    performance_metrics = performance_metrics.set_index("error_metric")

    # Performance Evaluation
    error_metrics = perf_regression(y_train, y_test, y_pred_train, y_pred_test, x_train, x_test)
    performance_metrics = pd.concat([performance_metrics, error_metrics], axis=0)
    performance_metrics["train"] = performance_metrics["train"].astype("object")

    if verbose:
        print("-----------------")
        print(f'The model score using K-fold cross validation (k={cv}) is {round(np.mean(val_scores), 3)} '
              f'with a standard deviation of {round(np.std(val_scores), 3)}')
        print("-----------------")
        print("The performance metrics of the model")
        display(performance_metrics)
        print("-----------------")

    if plot:
        
        regression_plots(prediction_results["train"], set_id="train", fig_size=(15, 5))
        regression_plots(prediction_results["test"], set_id="test", fig_size=(15, 5))

    return {"model": model, "val_scores": val_scores, "prediction_results": prediction_results,
            "performance_metrics": performance_metrics}

## Plotting

In [ ]:
# Function to plot a correlation map

def my_correlation_heatmap(df, figsize = (16,16)):
    """
    A function to plot a correlation heatmap from a DataFrame.

    Parameters
    ----------
    df : DataFrame
        The target DataFrame
    figsize : Tuple, optional
        Tuple seting the figure size. The default is (16,16).

    Returns
    -------
    None.

    """

    # correlation matrix

    correlation_matrix = df.corr()

    # create figure and axes
    fig, ax = plt.subplots(figsize = figsize)

    # set title
    ax.set_title('Correlation Heatmap', fontweight='bold')


    sns.heatmap(correlation_matrix,  # the data for the heatmap
                annot=True,  # show the actual values of correlation
                cmap='seismic',  # provide the 'seismic' colormap
                center=0,  # specify the value at which to center the colormap
                )

## Utility Functions


In [ ]:
# Function to store in a dictionary the number of nan values per column

def nan_counter(df):
    """
    

    Parameters
    ----------
    df : DataFrame
        The target DataFrame.

    Returns
    -------
    remaining_nan : Dictionary
        A dictionary with the following structure. Key is the df column, value 
        is the number of null values in the corresponding column.

    """
    
    remaining_nan = {}

    for column in df.columns:

        remaining_nan[column] = df[column][df[column].isna() == True].size

    return remaining_nan

# Initial Data Exploration


Let us load the data using the user defined function "load_data()"

In [ ]:
hp_df = load_data("Data_MidTerm_Project_Real_State_Regression.xls", source_type="excel")


We standarize the headings of the columns using user defined function "standard_headings()".

In [ ]:
hp_df = standard_headings(hp_df)


In [ ]:
hp_df

We do a first check of the data using methods "info()" and "describe()."



In [ ]:
hp_df.info()

In [ ]:
hp_df.describe()

We check also the DataFrame shape. We have 21,597 records and 21 features.



In [ ]:
hp_df_shape = hp_df.shape
hp_df_shape

We will also check the number of null values in all columns using the user defined function "nan_counter()."



In [ ]:
nan_counter(hp_df)


We drop duplicates records and check the new df size. There are no duplicates records.



In [ ]:
hp_df.drop_duplicates().shape

## Price Analysis

The house price is our dependent variable. Let us create a dedicated variable for easy access and some price analysis.



In [ ]:
price = hp_df["price"]

### Description and distribution

In [ ]:
# Descriptive statistics summary
price.describe()

In [ ]:
# Histogram

# create figure and axes
fig, ax = plt.subplots(figsize = (10,6))

sns.distplot(price);

In [ ]:
# Skewness and Kurtosis
print(f"Skewness: {price.skew()}")
print(f"Kurtosis: {price.kurt()}")

The price is not following a normal distribution, with a high positive skewness and Kurtosis as a result of the very 'long' right tail. We will have therefore a significant number of 'outliers' in a boxplot. We will need to figure out if the outliers should be removed or if they are required to describe the data.

In [ ]:
# Boxplot

# create figure and axes
fig, ax = plt.subplots(figsize = (10,6))

sns.boxplot(data=price, x=price);

### Feature Correlations with Price


In [ ]:
my_correlation_heatmap(hp_df, figsize=(16,16))

Let us plot a reduced version of the heatmap only with the 10 more correlated features.



In [ ]:
price_corr_sorted = hp_df.corr()["price"].sort_values(ascending=False)
top_10_price_corr = list(price_corr_sorted.index[0:11])
top_10_price_corr

In [ ]:
my_correlation_heatmap(hp_df[top_10_price_corr], figsize=(8,8))


The predictors with the highest correlation with the price are sqft_living (living area) and grade (building quality). Let us take a deeper look into these two.

It is also interesting the high correlation between sqft_linving and the other features related with area. We will drop all except sqft_living.

For the price we can use a simple scatter plot. It is obvious that there is a relatively strong linear correlation between price an sqft_living. We can also see that the houses with very high prices (outliers) have in general also very high living areas.

In [ ]:
# Scatter plot with sqft_living
var = 'sqft_living'
data = pd.concat([price, hp_df[var]], axis=1)

fig, ax = plt.subplots(figsize = (8,8))
sns.regplot(x=var, y="price", data=data, scatter_kws={"color": "red"}, line_kws={"color": "black"});

'Grade' is a categorical variable. A boxplot is more suited therefore than a scatter plot. We can observe an interesting exponential relation between price an grade. Again the very expensive houses have in general very high grades.

In [ ]:
#box plot grade/saleprice
var = 'grade'
data = pd.concat([price, hp_df[var]], axis=1)
fig, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="price", data=data);

It is intuitive than grade an condition are kind representing the same information. Let us plot boxplots of the condtion. It is clear (also in the correlation heatmap) that condition has not a significan relation with the price. We will drop it.

In [ ]:
#box plot condition/saleprice
var = 'condition'
data = pd.concat([price, hp_df[var]], axis=1)
fig, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="price", data=data);

We plot also boxplots for the number of bedrooms and bathrooms.



In [ ]:
#box plot n bathrooms/saleprice
var = 'bathrooms'
data = pd.concat([price, hp_df[var]], axis=1)
fig, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="price", data=data)

In [ ]:
#box plot n bedrooms/saleprice
var = 'bedrooms'
data = pd.concat([price, hp_df[var]], axis=1)
fig, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="price", data=data)

Another feature that could relevant is the yr_renovated (renovation year). Unfourtunately this features has many records with value 0. We will ned to deal with this.



In [ ]:
# Records with yr_renovated = 0

yr_renovated_n_0 = hp_df.yr_renovated.value_counts(dropna=False).sort_index()

print(f'The number of record with availabe info regarding the renovation year is {hp_df_shape[0] - yr_renovated_n_0[0]}.')

# Dropping features

In [ ]:
drop_columns = ["id", "date","sqft_lot", "sqft_above", "sqft_basement", "sqft_lot15", "condition"]

# id and date are useless for our analysis.
# sqft_lot, sqft_above, sqft_basement and sqft_lot15 are highly correlated with sqft_living and therefore redundant.
# the information contained in condition is already in grade and its correlation with price is very low. It can be dropped.

In [ ]:
hp_df = hp_df.drop(columns = drop_columns)


# Data transformation

## Building Age treatment

### Combine yr_built and yr_renovated into one single feature


What about if we combine the renovation year (with many zeros, possibly missing values) and the construction year into one single column. We will replace the construction year by the renovation year, for those records (very few) whit information regarding the renovation. This basically assumes that a building renovation equals to a new construction in terms of prices.

In [ ]:
yr_built = hp_df["yr_built"].copy()
yr_renovated = hp_df["yr_renovated"].copy()
yr_ren_built = yr_renovated.where(yr_renovated != 0, other = yr_built)
yr_ren_built

Let us now convert this new feature into a variable representing the age.



In [ ]:
max(yr_ren_built)

In [ ]:
age =  max(yr_ren_built) - yr_ren_built
age

In [ ]:
hp_df["age"] = age
hp_df = hp_df.drop(columns=["yr_renovated", "yr_built"])
hp_df

## Dropping row with 33 bedrooms


There is one record with 33 rooms that is obviously a typo. We will remove this row.



In [ ]:
hp_df = hp_df[hp_df["bedrooms"] != 33]
hp_df

## Location Variables - distto


Before starting the analysis, it is reasonable to expect that the location of the house has a significant impact on the house price. However the location estimators in the dataset (zipcode, long and lat) does not show a high correlation with the price. Can we engineer a new feature that better represent the house location?

In this section we will engineer a feature representing the disctance of each of the houses to a specific point, in our case to Seattle and two Bellevue. These two locations are hot spots, areas where the house prices are very high. To calculate the distance we will use the haversine function.

In [ ]:
latlong = hp_df.loc[:, ['lat', 'long']]
latlong_tuple = list(zip(latlong.loc[:, 'lat'], latlong.loc[:, 'long']))
latlong["lat_long"] = latlong_tuple
latlong

In [ ]:
dist_to_seattle = latlong.loc[:, 'lat_long'].apply(haversine, point2=(47.609395, -122.336283))

dist_to_bellevue = latlong.loc[:, 'lat_long'].apply(haversine, point2=(47.616492, -122.188985))

In [ ]:
hp_df.loc[:, 'dist_to_seattle'] = dist_to_seattle
hp_df.loc[:, 'dist_to_bellevue'] = dist_to_bellevue

The correlation heatmap shows a better correlation with price of these new two features.



In [ ]:
my_correlation_heatmap(hp_df, figsize=(16,16))

## Normalization - Price


Let us analyse how normal is the price distribution.



In [ ]:
# create figure and axes
fig, axs = plt.subplots(1,2,figsize = (12,6))

sns.distplot(price,fit=norm, ax=axs[0])
result = probplot(price, plot=plt);

It is clear there is some room for improvement. Let us normalize the price. A logarithmic transformation usually works fine with distribution with long one sided tails.



### Logarithmic Transformation


In [ ]:
price_log_norm = np.log(price)

In [ ]:
# create figure and axes
fig, axs = plt.subplots(1,2,figsize = (12,6))

sns.distplot(price_log_norm,fit=norm, ax=axs[0])
result = probplot(price_log_norm, plot=plt)

Now the price follows an almost perfect normal distribution.



## Normalization - sqft_living


We will do the same with our main numerical estimator, sqft_living.



In [ ]:
var = "sqft_living"

# create figure and axes
fig, axs = plt.subplots(1,2,figsize = (16,8))

sns.distplot(hp_df[var],fit=norm, ax=axs[0])
result = probplot(hp_df[var], plot=plt)

### Power Transformation


This time we will be using a box-cox transformation.



In [ ]:
sqft_living_transformer = PowerTransformer(method="box-cox").fit(hp_df[var].to_numpy().reshape(-1,1))
sqft_living_power_norm = sqft_living_transformer.transform(hp_df[var].to_numpy().reshape(-1,1))
sqft_living_power_norm = pd.DataFrame(sqft_living_power_norm)[0]

In [ ]:
# create figure and axes
fig, axs = plt.subplots(1,2,figsize = (16,8))

sns.distplot(sqft_living_power_norm,fit=norm, ax=axs[0])
result = probplot(sqft_living_power_norm, plot=plt)

The resultant distribution again follows very closely the normal one.



# Regression Models

## Model 1 - Simple Linear Regression (sqft_living VS Price)


This first model is a very simple one: a linear regression using only the main numerical estimator, sqft_living.



In [ ]:
# Pipeline

hp_df = (load_data("Data_MidTerm_Project_Real_State_Regression.xls", source_type="excel")
         .pipe(standard_headings)
         .pipe(remove_rows)
         .reset_index(drop=True)
         )

In [ ]:
x = hp_df["sqft_living"]
y = hp_df["price"]

In [ ]:
model1_transformed = my_transformations(x, y, test_size=0.2, numerical=True, scaler=None, categorical=False)

In [ ]:
x_train = pd.DataFrame(model1_transformed["train"][0])
y_train = model1_transformed["train"][3]

x_test = pd.DataFrame(model1_transformed["test"][0])
y_test = model1_transformed["test"][3]

In [ ]:
model1_output = regression(x_train, y_train, x_test, y_test, LinearRegression(), cv=10, verbose=True, plot=True, y_transformer=None)

## Model 2 - Multiple Linear Regression (all variables) - with statsmodels


In [ ]:
# Inputs for the Pipeline Controller

drop_columns = ["id", "date"]

In [ ]:
# Pipeline Control

hp_df = (load_data("Data_MidTerm_Project_Real_State_Regression.xls", source_type="excel")
         .pipe(standard_headings)
         .drop(drop_columns, axis=1)
         .pipe(remove_rows)
         .reset_index(drop=True)
        )

Model 2 considers all features (except id and date, which are irrelevant), no transformations and a linear regression.

In [ ]:
y = hp_df["price"]
x = hp_df.drop(["price"], axis=1)

In [ ]:
model2_transformed = my_transformations(x, y, test_size=0.2, numerical=True, scaler=None, categorical=True)


In [ ]:
x_train = pd.concat([model2_transformed["train"][0], model2_transformed["train"][1]], axis=1)
y_train = model2_transformed["train"][3]

x_test = pd.concat([model2_transformed["test"][0], model2_transformed["test"][1]], axis=1)
y_test = model2_transformed["test"][3]

In [ ]:
# Summary from statsmodels

x_train_const = sm.add_constant(x_train)  

model = sm.OLS(y_train, x_train_const).fit()

print(model.summary())

The hypothesis testing shows that sqft_lot and floors don't have an impact on the house price. The t value also shows that grade and sqft_living are the two more important features.

In [ ]:
model2_output = regression(x_train, y_train, x_test, y_test, LinearRegression(), cv=10, verbose=True, plot=True, y_transformer=None)


## Model 3 - Multiple Linear Regression - Reduced number of variables and Transformations


In [ ]:
# Inputs for the Pipeline Controller

drop_columns = ["id", "date","sqft_lot", "sqft_above", "sqft_basement", "sqft_lot15", "condition", "floors", "sqft_living15"]

In [ ]:

# Pipeline Control

hp_df = (load_data("Data_MidTerm_Project_Real_State_Regression.xls", source_type="excel")
                   .pipe(standard_headings)
                   .drop(drop_columns, axis=1)
                   .pipe(transform_renovated_built)  # Combination of yr_built and yr_renovated into one single feature
                   .pipe(remove_rows)
                   .reset_index(drop=True)
                   .pipe(normalizations)     # Price and sqft_living Normalization
                   .pipe(dist_to)            # Location Enigeneered Feature
                   )

Model 3 considers a reduced set of features, the discussed transformations and a linear regression.



In [ ]:
y = hp_df["price"]
x = hp_df.drop(["price"], axis=1)

In [ ]:
model3_transformed = my_transformations(x, y, test_size=0.2, numerical=True, scaler=None, categorical=True)


In [ ]:
x_train = pd.concat([model3_transformed["train"][0], model3_transformed["train"][1]], axis=1)
y_train = model3_transformed["train"][3]

x_test = pd.concat([model3_transformed["test"][0], model3_transformed["test"][1]], axis=1)
y_test = model3_transformed["test"][3]

In [ ]:
model3_output = regression(x_train, y_train, x_test, y_test, LinearRegression(), cv=10, verbose=True, plot=True, y_transformer="log")

## Model 4 - Ridge Regression

In [ ]:
# Inputs for the Pipeline Controller

drop_columns = ["id", "date","sqft_lot", "sqft_above", "sqft_basement", "sqft_lot15", "condition", "floors", "sqft_living15"]

In [ ]:
# Pipeline Control

hp_df = (load_data("Data_MidTerm_Project_Real_State_Regression.xls", source_type="excel")
                   .pipe(standard_headings)
                   .drop(drop_columns, axis=1)
                   .pipe(transform_renovated_built)  # Combination of yr_built and yr_renovated into one single feature
                   .pipe(remove_rows)
                   .reset_index(drop=True)
                   .pipe(normalizations)     # Price and sqft_living Normalization
                   .pipe(dist_to)            # Location Enigeneered Feature
                   )

Model 4 considers a reduced set of features, the discussed transformations and a ridge regression.

In [ ]:
y = hp_df["price"]
x = hp_df.drop(["price"], axis=1)

In [ ]:
model4_transformed = my_transformations(x, y, test_size=0.2, numerical=True, scaler=None, categorical=True)


In [ ]:
x_train = pd.concat([model4_transformed["train"][0], model4_transformed["train"][1]], axis=1)
y_train = model4_transformed["train"][3]

x_test = pd.concat([model4_transformed["test"][0], model4_transformed["test"][1]], axis=1)
y_test = model4_transformed["test"][3]

In [ ]:
model4_output = regression(x_train, y_train, x_test, y_test, Ridge(alpha=100), cv=10, verbose=True, plot=True, y_transformer="log")


Ridge is not bringing any improvement. Kind of expected, the model is not too complex, on the contrary is lacking complexity.



## Model 5 - Lasso Regression


In [ ]:
# Inputs for the Pipeline Controller

drop_columns = ["id", "date","sqft_lot", "sqft_above", "sqft_basement", "sqft_lot15", "condition", "floors", "sqft_living15"]

In [ ]:
# Pipeline Control

hp_df = (load_data("Data_MidTerm_Project_Real_State_Regression.xls", source_type="excel")
                   .pipe(standard_headings)
                   .drop(drop_columns, axis=1)
                   .pipe(transform_renovated_built)  # Combination of yr_built and yr_renovated into one single feature
                   .pipe(remove_rows)
                   .reset_index(drop=True)
                   .pipe(normalizations)     # Price and sqft_living Normalization
                   .pipe(dist_to)            # Location Enigeneered Feature
                   )

Model 5 considers a reduced set of features, the discussed transformations and a Lasso regression.



In [ ]:

y = hp_df["price"]
x = hp_df.drop(["price"], axis=1)


In [ ]:
model5_transformed = my_transformations(x, y, test_size=0.2, numerical=True, scaler=None, categorical=True)


In [ ]:
x_train = pd.concat([model5_transformed["train"][0], model5_transformed["train"][1]], axis=1)
y_train = model5_transformed["train"][3]

x_test = pd.concat([model5_transformed["test"][0], model5_transformed["test"][1]], axis=1)
y_test = model5_transformed["test"][3]

In [ ]:
model5_output = regression(x_train, y_train, x_test, y_test, Lasso(alpha=1), cv=10, verbose=True, plot=True, y_transformer="log")


Lasso is not working properly, unless we use ver low alphas. Makes sense, we have very few features and Lasso even reducing some more.



## Model 6 - Polynomial Regression


In [ ]:
# Inputs for the Pipeline Controller

drop_columns = ["id", "date","sqft_lot", "sqft_above", "sqft_basement", "sqft_lot15", "condition", "floors", "sqft_living15"]

In [ ]:
# Pipeline Control

hp_df = (load_data("Data_MidTerm_Project_Real_State_Regression.xls", source_type="excel")
                   .pipe(standard_headings)
                   .drop(drop_columns, axis=1)
                   .pipe(transform_renovated_built)  # Combination of yr_built and yr_renovated into one single feature
                   .pipe(remove_rows)
                   .reset_index(drop=True)
                   .pipe(normalizations)     # Price and sqft_living Normalization
                   .pipe(dist_to)            # Location Enigeneered Feature
                   )

Model 6 considers a reduced set of features, the discussed transformations and a polynomial regression of order 2.



In [ ]:
y = hp_df["price"]
x = hp_df.drop(["price"], axis=1)

In [ ]:
model6_transformed = my_transformations(x, y, test_size=0.2, numerical=True, scaler=None, categorical=True)


In [ ]:
x_train = pd.concat([model6_transformed["train"][0], model6_transformed["train"][1]], axis=1)
y_train = model6_transformed["train"][3]

x_test = pd.concat([model6_transformed["test"][0], model6_transformed["test"][1]], axis=1)
y_test = model6_transformed["test"][3]

In [ ]:
# Polynomial Transformation - order n

n = 2
polynomial_features= PolynomialFeatures(degree=n)
x_train_poly = polynomial_features.fit_transform(x_train)
x_test_poly = polynomial_features.transform(x_test)

In [ ]:
model6_output = regression(x_train_poly, y_train, x_test_poly, y_test, LinearRegression(), cv=10, verbose=True, plot=True, y_transformer="log")


## Model 7 - XGBoost


In [ ]:
# Inputs for the Pipeline Controller

drop_columns = ["id", "date","sqft_lot", "sqft_above", "sqft_basement", "sqft_lot15", "condition", "floors", "sqft_living15"]

In [ ]:
# Pipeline Control

hp_df = (load_data("Data_MidTerm_Project_Real_State_Regression.xls", source_type="excel")
                   .pipe(standard_headings)
                   .drop(drop_columns, axis=1)
                   .pipe(transform_renovated_built)  # Combination of yr_built and yr_renovated into one single feature
                   .pipe(remove_rows)
                   .reset_index(drop=True)
                   .pipe(normalizations)     # Price and sqft_living Normalization
                   .pipe(dist_to)            # Location Enigeneered Feature
                   )

Model 7 considers a reduced set of features, the discussed transformations and a Extreme Gradient Boost model.



In [ ]:
y = hp_df["price"]
x = hp_df.drop(["price"], axis=1)

In [ ]:
model7_transformed = my_transformations(x, y, test_size=0.2, numerical=True, scaler=None, categorical=True)

In [ ]:
x_train = pd.concat([model7_transformed["train"][0], model7_transformed["train"][1]], axis=1)
y_train = model7_transformed["train"][3]

x_test = pd.concat([model7_transformed["test"][0], model7_transformed["test"][1]], axis=1)
y_test = model7_transformed["test"][3]

In [ ]:
model7_output = regression(x_train, y_train, x_test, y_test, xgboost.XGBRegressor(n_estimators=90, learning_rate=0.1, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=6), cv=10, verbose=True, plot=True, y_transformer="log")

# Conclusion


- An Extreme Grandient Boosting (XGB) model with an adjusted R2=0.9 has been developed to prediced the price of houses in Kings County Area, Seattle.
- The most revelant estimators are the living area (sqft_living) and the building grade (quality).
- This high performance has been obtained using only 12 estimators.
- Normalization of the price and the living area demonstrated to be very powerful in this dataset, bringing a 4% improvement on the adjusted R2
- The Location Engineered Features brought a 6% improvement on the adjusted R2. These feature are based on the dataset longitud and latitud data and represent the distance of each house to two hot-spots (areas with very high prices), belleveu and downtown Seattle
- There are a lot of outliers in the data (houses with very prices). However it makes sense to keep them as they are part of the nature of the data. There are house with very high prices on the market but with very specific characteristics: huge living space, water access, very high grade, very nice locations